In [7]:
import pandas as pd
import numpy as np

In [8]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [9]:
df = pd.read_csv("df.csv")

,date,Unnamed: 0,id,serial,last_modified_utc,i_power,i_boostpower,i_meterfeed,i_metercons,i_temp1,...,i_temp4,exported,power_consumption,on_off,thermal_consumption,thermal_consumption_kwh,mean_temperature,kwh_eq_state,demand_price,feedin_price
0,2022-01-01 00:15:00,29004,72896886,2.001000e+15,1/1/2022 0:15,4,0,0,-2,557.0,...,0.0,0,0.0,1,0.000000,0.000000,47.066667,62.786933,0.40,0.133333
1,2022-01-01 00:30:00,29005,72899258,2.001000e+15,1/1/2022 0:30,0,0,0,0,556.0,...,0.0,0,0.0,0,-0.333333,-0.444667,46.800000,62.431200,0.40,0.133333
2,2022-01-01 00:45:00,29006,72902996,2.001000e+15,1/1/2022 0:46,0,0,0,0,555.0,...,0.0,0,0.0,0,-0.433333,-0.578067,46.366667,61.853133,0.40,0.133333
3,2022-01-01 01:00:00,29007,72904623,2.001000e+15,1/1/2022 1:00,0,0,0,0,554.0,...,0.0,0,0.0,0,-0.100000,-0.133400,46.266667,61.719733,0.30,0.100000
4,2022-01-01 01:15:00,29008,72908836,2.001000e+15,1/1/2022 1:16,0,0,0,0,553.0,...,0.0,0,0.0,0,-0.033333,-0.044467,46.233333,61.675267,0.30,0.100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21809,2022-08-18 22:45:00,50813,152870777,2.001000e+15,8/18/2022 22:45,0,0,0,0,560.0,...,0.0,0,0.0,0,-0.033333,-0.044467,48.800000,65.099200,0.50,0.166667
21810,2022-08-18 23:00:00,50814,152876427,2.001000e+15,8/18/2022 23:01,0,0,0,0,559.0,...,0.0,0,0.0,0,-0.066667,-0.088933,48.733333,65.010267,0.35,0.116667
21811,2022-08-18 23:15:00,50815,152879638,2.001000e+15,8/18/2022 23:15,0,0,0,0,558.0,...,0.0,0,0.0,0,-0.066667,-0.088933,48.666667,64.921333,0.35,0.116667
21812,2022-08-18 23:30:00,50816,152881531,2.001000e+15,8/18/2022 23:30,0,0,0,0,557.0,...,0.0,0,0.0,0,-0.066667,-0.088933,48.600000,64.832400,0.35,0.116667


In [11]:
print(df.kwh_eq_state.max())

82.53013333333332


In [685]:
max_storage_tank = 82.53
optimum_storage = 50.00
gamma = 0.05

In [686]:
def step(s, thermal_consumption, a):
    '''
    action = [0; 82.53]
    '''
    cool_down = 0.1 
    heat_increase = a - cool_down
    s_1 = s + heat_increase - thermal_consumption
    
    if s_1 > max_storage_tank:
        s_1 = max_storage_tank
        
    return s_1

In [687]:
def reward(action, pv_excess, demand_price, feedin_price, power_consumption, thermal_consumption, state):
    
    kwh_increase = action
    
    if state+kwh_increase > max_storage_tank:
        kwh_increase = max_storage_tank - state
    
    '''
    
    ! VORZEICHEN DER VARIABLEN !
    
    '''
    diff = pv_excess - kwh_increase - power_consumption - thermal_consumption    
        
    if diff < 0:
        reward = -(diff * demand_price + (kwh_increase - diff) * feedin_price)
    else:
        reward = -(kwh_increase * feedin_price)  + diff * feedin_price
        
    add_penalty=10
    if state < 40.00:
        reward= reward -add_penalty
        
    '''
    PENALTY FOR DISTANCE TO OPTIMUM
    '''
    reward -= abs(state - optimum_storage)*gamma
    
    
    return reward

Develop Tree Storage

In [688]:
levels = 19

tree = np.array([0,11,13,2,8,4,20,3,8,20,11,5,4,10,9])


tree = np.zeros(2**(levels+1)-1)


tree[::2] =1
tree = np.insert(tree, 0, 0)
tree = np.insert(tree, 0, 0)



res_sum = np.zeros(len(tree))
states = np.zeros(len(tree))
states[0] = 41.00

tree, len(tree)

(array([0., 0., 1., ..., 1., 0., 1.]), 1048577)

In [689]:
nodes = 1
run = 0
for i in range(levels):    
    print(i,run)
    for n in range(nodes):
        
        i = run
        root = tree[i]
        result_root = res_sum[i]
        state_root = states[i]
        
        
        left = tree[i*2 +1]
        right = tree[i*2 +2]
        
        
        res_sum[i*2 +1] = result_root + reward(left, 2.5,0.4,0.1, 0.1,0.1, state_root)
        res_sum[i*2 +2] = result_root + reward(right,2.5 ,0.4,0.1, 0.1,0.1, state_root)
        
        '''
        step(s, thermal_consumption, a):
        '''
        
        states[i*2 +1] = step(state_root, 0.1, left)
        states[i*2 +2] = step(state_root, 0.1, right)
        
        
        run +=1
        
    nodes *= 2

0 0
1 1
2 3
3 7
4 15
5 31
6 63
7 127
8 255
9 511
10 1023
11 2047
12 4095
13 8191
14 16383
15 32767
16 65535
17 131071
18 262143


In [690]:
leaves = res_sum[262143:-3]


print(leaves.argmax(), len(leaves))

b_i = leaves.argmax() 
leaves.max()

786240 786431


-0.9000000000000173

In [691]:
def backtrack_seq(tree, idx, levels):
    seq = []
    for i in range(levels):
        action_tmp = tree[int(idx)]
        
        i_add = -1 if int(action_tmp) == 0 else -2
        
        idx = idx/2 + i_add
        a = tree[int(idx)]
        seq.append(a)
    
    return seq

In [692]:
seq_ = backtrack_seq(tree, b_i+262143 , levels)
print("sum rewards: ",leaves[b_i])
print("idx: ", b_i)
print(seq_, sum(seq_))
print(states[b_i+262143])


sum rewards:  -0.9000000000000173
idx:  786240
[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0] 12.0
49.199999999999946


In [693]:
res_sum

array([ 0.  , -0.22, -0.42, ..., -3.24,  0.  ,  0.  ])

In [695]:
x = 12
41+x*0.8-(19-x)*0.2

49.2

<h1>Backup Base Tree Storage</h1>

In [524]:
levels = 3

tree = np.array([0,11,13,2,8,4,20,3,8,20,11,5,4,10,9])
tree = np.ones(2**4-1)

res_sum = np.zeros(len(tree))


In [525]:
nodes = 1
run = 0
for i in range(levels):    
    for n in range(nodes):
        
        i = run
        root = tree[i]
        
        
        left = tree[i*2 +1]
        right = tree[i*2 +2]

        result_root = res_sum[i]


        #print(i, root, left,right)
        

        res_sum[i*2 +1] = result_root + left
        res_sum[i*2 +2] = result_root + right
        
        run +=1
        
    nodes *= 2

In [526]:
print(res_sum.shape)
res_sum[:100]

(15,)


array([0., 1., 1., 2., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3.])

In [463]:
def backtrack_seq(tree, idx, levels):
    seq = []
    for i in range(levels):
        action_tmp = tree[int(idx)]
        
        i_add = -1 if int(action_tmp) == 0 else -2
        
        idx = idx/2-1
        a = tree[int(idx)]
        seq.append(a)
    
    return seq

In [464]:
print(backtrack_seq(tree, 14 , 3))


[20, 13, 0]


[1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [684]:
np.ones(2**50)

MemoryError: Unable to allocate 8.00 PiB for an array with shape (1125899906842624,) and data type float64